# Load packages that we will need

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
import json

# Categorical columns
The organisers of the competition provided columns that are categorical. let's make a list containing those columns and map them to str type when loading pandas dataset

In [2]:
def create_col_name(base_str, start_int, end_int):
    return [f'{base_str}{i}' for i in range(start_int, end_int+1)]

In [3]:
cat_cols = (['ProductCD'] + create_col_name('card', 1, 6) + ['addr1', 'addr2', 'P_emaildomain', 'R_emaildomain'] + 
            create_col_name('M', 1, 9) + ['DeviceType', 'DeviceInfo'] + create_col_name('id_', 12, 38))

id_cols = ['TransactionID']

dep_var = 'isFraud'

In [4]:
type_map = {c: str for c in cat_cols + id_cols}

In [5]:
table_names = ['train_identity', 'train_transaction', 'test_identity', 'test_transaction']
tables = [pd.read_csv(f'data/{fname}.csv', dtype=type_map, low_memory=False) for fname in table_names]
df_train_id, df_train_trans, df_test_id, df_test_trans = tables

In [6]:
df_train_id.shape, df_train_trans.shape, df_test_id.shape, df_test_trans.shape

((144233, 41), (590540, 394), (141907, 41), (506691, 393))

In [7]:
df_train_id.head(5)

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [8]:
df_train_trans.head(5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Join the datasets by the TransactionID

In [9]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [10]:
df_train = join_df(df_train_trans, df_train_id, id_cols)

In [11]:
assert (df_train.shape[0] == df_train_trans.shape[0]) & (df_train.shape[1] == df_train_trans.shape[1]+ df_train_id.shape[1]-1)

# Data cleaning and Feature engineering
Upon looking at the test dataset, some columns have no missing values while in the dataset there are missing values for those columns. we want to make sure the model will not capitalize on columns that we know will have a complete different distribution in the test set. Let's add random NaN values on the Columns according the the percentage of NaN distribution in test dataset

In [12]:
C_NaN_Percent_dict = {'C1': 0.000592076827889187, 'C2': 0.000592076827889187, 'C3': 0.000592076827889187, 'C4': 0.000592076827889187, 'C5': 0.000592076827889187, 'C6': 0.000592076827889187,\
 'C7': 0.000592076827889187, 'C8': 0.000592076827889187, 'C9': 0.000592076827889187, 'C10': 0.000592076827889187, 'C11': 0.000592076827889187, 'C12': 0.000592076827889187,\
 'C13': 0.9370602596059532, 'C14': 0.000592076827889187} 
C_NaN_Percent_dict = { key: round(val*100,2) for key,val in C_NaN_Percent_dict.items()}

In [13]:
train_size = df_train.shape[0]
for key,val in C_NaN_Percent_dict.items():
    rate = int(train_size * val/100)
    rate = rate if rate < 50 else 50
    sample_idx = list(df_train[key].sample(rate).index)
    print(key,len(sample_idx))
    df_train.iloc[sample_idx,df_train.columns.get_loc(key)] = np.nan

C1 50
C2 50
C3 50
C4 50
C5 50
C6 50
C7 50
C8 50
C9 50
C10 50
C11 50
C12 50
C13 50
C14 50


The Id columns in the training set and the test set are named differently. let's fix that.

In [15]:
col_name_map = {'id-{:02d}'.format(c):'id_{:02d}'.format(c) for c in range(1, 39)}

In [16]:
df_test_id.rename(columns=col_name_map,inplace=True)

In [17]:
col_type_map = {f'id_{c}':str for c in range(12, 39)}
df_test_id = df_test_id.astype(col_type_map)

In [18]:
df_test = join_df(df_test_trans, df_test_id, id_cols)

In [19]:
assert (df_test.shape[0] == df_test_trans.shape[0]) & (df_test.shape[1] == df_test_trans.shape[1]+ df_test_id.shape[1]-1)

We saw that the dataset was split by time. the test and the training cover different timestamp. this indicates that time is of importance to this exercise. let's add features called day and hour form the time stamp difference. notice that this is not the actual day of the week or the hour of the week but the periodicity of days of the week and the hours of the day will be preserved.

In [20]:
df_train['day'] = ((df_train['TransactionDT']//(3600*24)-1)%7)+1
df_test['day'] = ((df_test['TransactionDT']//(3600*24)-1)%7)+1
df_train['hour'] = ((df_train['TransactionDT']//3600)%24)+1
df_test['hour'] = ((df_test['TransactionDT']//3600)%24)+1

some of the id_* columns have strange values called 'nan'. let's replace this with np.nan

In [21]:
df_train.loc[df_train['id_31']=='nan','id_31'].shape,df_test.loc[df_test['id_31']=='nan','id_31'].shape

((0,), (5282,))

As per recommendation from EDA, let's change all values that are string 'nan' to 'np.nan'

In [22]:
for i in range(1, 39):
    c = 'id_{:02d}'.format(i)
    df_train.loc[df_train[c]=='nan',c] = np.nan
    df_test.loc[df_test[c]=='nan',c] = np.nan

In [23]:
df_train.loc[df_train['id_31']=='nan','id_31'].shape,df_test.loc[df_test['id_31']=='nan','id_31'].shape

((0,), (0,))

The column id_31 has information about device used by the client. this information include the version number. However given that our dataset is divided by timestamp, it makes sense that version used in the past will no longer be available in the future. this implies that the column might become not usable for future data. to keep it useful, let's separate device name from it's version by adding a new column for versions

In [24]:
def remove_numbers(df_train, df_test, feature):
    df_train.loc[:, f'{feature}_v'] = df_train[feature].str.replace(r'[^0-9\.0-9]', '', regex=True)
    df_train.loc[:, feature] = df_train[feature].str.replace(r'[^A-Za-z]', '', regex=True)
    df_train.loc[:, feature].astype(str)
    df_test.loc[:, f'{feature}_v'] = df_test[feature].str.replace(r'[^0-9\.0-9]', '', regex=True)
    df_test.loc[:, feature] = df_test[feature].str.replace(r'[^A-Za-z]', '', regex=True)
    df_test.loc[:, feature].astype(str)

In [25]:
remove_numbers(df_train, df_test, 'id_31')

In [26]:
df_train['id_31_v'].unique()

array([nan, '6.2', '11.0', '62.0', '15.0', '', '49.0', '61.0', '16.0',
       '14.0', '56.0', '57.0', '54.0', '60.0', '10.0', '9.0', '59.0',
       '4.0', '55.0', '52.0', '46.0', '58.0', '48.0', '13.0', '53.0',
       '8.0', '7.0', '532', '50.0', '51.0', '63.0', '5.4', '47.0', '531',
       '43.0', '5.2', '200', '64.0', '44', '4.2', '6.4', '65.0', '17.0',
       '66.0', '3.3', '67.0', '69.0'], dtype=object)

In [27]:
df_train.to_csv('data/train.csv',index=False)

In [28]:
df_test.to_csv('data/test.csv',index=False)

In [29]:
numeric_cols = [col for col in df_train.columns.tolist() if col not in cat_cols + id_cols + [dep_var]]

In [30]:
features = {}
features['cat'] = cat_cols
features['cont'] = numeric_cols
features['dep_var'] = dep_var
#save the best params to a file
with open('artifacts/features.txt', 'w') as outfile:
    json.dump(features, outfile)

# Save a small stratified sample to use for quick testing of our code when writing our model

In [31]:
def stratifiedSample(df,n_samples,splitter):
    cols = list(df.columns.values)
    cols.remove(splitter)
    X = df[cols]
    y = df[splitter]

    splits = StratifiedShuffleSplit(n_splits=1, test_size=n_samples, random_state=42)
   
    for _, test_index in splits.split(X, y):
        sample = df.iloc[test_index]


    return sample

In [32]:
df_train_sample = stratifiedSample(df_train,41326,dep_var)

In [33]:
df_train_sample.groupby([dep_var]).count()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,day,hour,id_31_v
isFraud,,,,,,,,,,,,,,,,,,,,,
0,39880,39880,39880,39880,39880,39326,39777,39776,39608,39777,...,5195,9074,9074,9074,9074,9066,7695,39880,39880,9027
1,1446,1446,1446,1446,1446,1417,1443,1442,1432,1443,...,261,799,799,799,799,798,604,1446,1446,799


In [34]:
df_train_sample.to_csv('data/train-sample.csv',index=False)

# Balanced samples
Our dataset is heavily imbalanced. let's adopt a strategy where we train on all the positive cases and then for the negative sample let's take three times the number of positive samples but order by reverse time. that's we take data starting from the latest timestamp in the training set.

Also since we added a small percentage of random NaN values on the C(1-12,14), we need to make sure these values will make it to the training set

In [35]:
def balanceSample(df):
    period = 3600*24*120
    missing = df[(df['V279'].isnull()) | (df['C1'].isnull()) | (df['C2'].isnull())  | (df['C3'].isnull())  | (df['C4'].isnull())  | (df['C5'].isnull())\
                 | (df['C6'].isnull()) | (df['C7'].isnull()) | (df['C8'].isnull()) | (df['C9'].isnull()) | (df['C10'].isnull()) | (df['C11'].isnull())\
                 | (df['C12'].isnull()) | (df['C14'].isnull())].copy()
    
    df = df[df['TransactionDT'] >= df['TransactionDT'].max()-period]
    pos = df[df['isFraud']==1]
    neg = df[df['isFraud']==0]
    neg = neg.sort_values(by=['TransactionDT'])
    count_pos = pos.shape[0]
    neg_sample = neg.sample(count_pos*3)
    sample = pd.concat([neg_sample,pos,missing])
    sample = sample.drop_duplicates(keep='last')
    
    sample = sample.sample(frac=1).reset_index(drop=True)
    print(f'Shape : {sample.shape}')
    return sample

In [36]:
df_train_balance = balanceSample(df_train)

Shape : (54249, 437)


In [37]:
df_train_balance.to_csv('data/train-balance.csv',index=False)

In [38]:
import gc
gc.collect()

0